In [1]:
import os
import pandas as pd
from scipy.integrate import odeint
from scipy.interpolate import interp1d

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import time
import matplotlib.pyplot as plt
import sys

sys.path.append("..")
from symbolic_RLC import fxu_ODE, fxu_ODE_mod, A_nominal, B_nominal
from torchid.ssfitter import  NeuralODE
from torchid.ssmodels import NeuralStateSpaceModelLin, NeuralStateSpaceModel

In [2]:
# In[Linearization time - 1 a time]
nx = 2
nu = 1

ss_model = NeuralStateSpaceModel(n_x=nx, n_u=nu, n_feat=64) #NeuralStateSpaceModelLin(A_nominal*Ts, B_nominal*Ts)
res_load = ss_model.load_state_dict(torch.load(os.path.join("models", "model_ARX_FE_sat_nonoise.pkl")))

In [3]:
VAR = []
for idx_var in range(nx):
    var = np.zeros((1,nx)).astype(np.float32)
    var[0,idx_var] = 1.0 # differentiate w.r.t the nth variable
    VAR.append(torch.tensor(var))

In [4]:
# Random linearization point
x_arr = np.random.rand(nx).astype(np.float32)
x_torch = torch.tensor(x_arr, requires_grad=True)

u_batch = np.random.rand(nu).astype(np.float32)
u_torch = torch.tensor(u_batch, requires_grad=True)


In [5]:
%%timeit -n 1000

# Linearization around the random point 

F_xu = ss_model(x_torch,u_torch)
A = np.empty((nx,nx))
B = np.empty((nx,nu))

for idx_var in range(nx):
    var = VAR[idx_var]
    #var = np.zeros((1,nx)).astype(np.float32)
    #var[0,idx_var] = 1.0 # differentiate w.r.t the nth variable
    F_xu.backward(var, retain_graph=True)
    A[idx_var,:] = np.array(x_torch.grad)
    B[idx_var,:] = np.array(u_torch.grad)
    x_torch.grad.data.zero_()
    u_torch.grad.data.zero_()

339 µs ± 6.06 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [6]:
# Random batch of linearization point

batch_size = 128
x_batch = np.random.rand(batch_size, nx).astype(np.float32)
x_torch = torch.tensor(x_batch, requires_grad=True)

u_batch = np.random.rand(batch_size, nu).astype(np.float32)
u_torch = torch.tensor(u_batch, requires_grad=True)

    
    
VAR = []
for idx_var in range(nx):
    var = np.zeros((batch_size,nx)).astype(np.float32)
    var[:,idx_var] = 1.0 # differentiate w.r.t the nth variable
    VAR.append(torch.tensor(var))



In [10]:
%%timeit -n 1000

# In[Linearization time - batched]
    
F_xu = ss_model(x_torch,u_torch)
A = np.empty((batch_size,nx,nx))
B = np.empty((batch_size,nx,nu))
for idx_var in range(nx):
    var = VAR[idx_var]
    F_xu.backward(var, retain_graph=True)
    A[:,idx_var,:] = np.array(x_torch.grad)
    B[:,idx_var,:] = np.array(u_torch.grad)
    x_torch.grad.data.zero_()
    u_torch.grad.data.zero_()

399 µs ± 41 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [13]:
# In[Linearization time - batched]
    
F_xu = ss_model(x_torch,u_torch)
A = np.empty((batch_size,nx,nx))
B = np.empty((batch_size,nx,nu))
for idx_var in range(nx):
    var = VAR[idx_var]
    F_xu.backward(var, retain_graph=True)
    A[:,idx_var,:] = np.array(x_torch.grad)
    B[:,idx_var,:] = np.array(u_torch.grad)
    x_torch.grad.data.zero_()
    u_torch.grad.data.zero_()

In [14]:
A

array([[[ 1.01585616e-03,  7.58396864e-01],
        [-3.85324424e-03, -7.05133379e-03]],

       [[ 1.01585616e-03,  7.58396864e-01],
        [-3.85324424e-03, -7.05133379e-03]],

       [[-4.45739087e-03,  7.95587599e-01],
        [-6.06386876e-03,  7.96987116e-03]],

       [[ 1.01585616e-03,  7.58396864e-01],
        [-3.85324424e-03, -7.05133379e-03]],

       [[ 1.01585616e-03,  7.58396864e-01],
        [-3.85324424e-03, -7.05133379e-03]],

       [[ 1.01585616e-03,  7.58396864e-01],
        [-3.85324424e-03, -7.05133379e-03]],

       [[ 1.01585616e-03,  7.58396864e-01],
        [-3.85324424e-03, -7.05133379e-03]],

       [[ 1.01585616e-03,  7.58396864e-01],
        [-3.85324424e-03, -7.05133379e-03]],

       [[ 1.01585616e-03,  7.58396864e-01],
        [-3.85324424e-03, -7.05133379e-03]],

       [[ 1.01585616e-03,  7.58396864e-01],
        [-3.85324424e-03, -7.05133379e-03]],

       [[ 3.95481475e-05,  7.42230892e-01],
        [-4.10071807e-03, -1.11490637e-02]],

       [[ 